In [30]:
import pandas as pd
from datetime import timedelta, date
from get_data import daterange, get_data, parse_info
import urllib2

In [57]:
cities = pd.read_csv('data/ongoing_cities.csv', header = False)

In [7]:
cities = cities[(cities['city']=='San Francisco')]

In [26]:
city = cities.city_code.values[0]

In [27]:
city

'USA-SFO-SNF'

In [3]:
url = 'https://s3-us-west-2.amazonaws.com/hoodsjson/USA-CA/USA-SFO-SNF/2014-09-30/0.html'

In [4]:
json_df = pd.read_json(url)    

In [5]:
condition = json_df['source'] == 'CRAIG'
craig_df = json_df[condition]
craig_df = craig_df.reset_index()
del craig_df['index']

In [6]:
cols = ['heading', 'body', 'price', 'lat', 'long', 'region', 'neighborhood', 
        'beds', 'baths', 'parking', 'washer_dryer']

In [7]:
results_df = parse_info(craig_df, headers = cols)

In [9]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498 entries, 0 to 497
Data columns (total 12 columns):
heading         498 non-null object
body            498 non-null object
price           495 non-null float64
lat             498 non-null object
long            498 non-null object
region          497 non-null object
neighborhood    490 non-null object
beds            497 non-null object
baths           465 non-null object
parking         498 non-null object
washer_dryer    498 non-null object
date            498 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(10)
memory usage: 50.6+ KB


In [47]:
sfdf = pd.read_csv('data/csvs/USA-SFO-SNF_USA-CA.csv', header=False)

In [53]:
sfdf.shape

(227756, 13)

In [51]:
len(sfdf.id.unique())

185130

In [56]:
sfdf.region.value_counts()

sfc    208587
nby      8421
sby      5727
eby      2243
scz      1187
pen      1056
grg        14
gbs        13
lgb        13
csd         9
oah         4
chc         3
wst         3
lac         3
mnh         2
mdc         2
mlt         1
see         1
hnp         1
clk         1
wvl         1
doc         1
dtype: int64

In [73]:
sfdf.body[9]

'\n Elegant and Charming Two Bedroom Corner Unit!\n 297 18th Ave, San Francisco, CA, 94121\n Schedule a Tour: http://www.apartmentsrm.com/properties/2025/?source=C\n Contact Us: http://www.apartmentsrm.com/tenant/contact/2025?source=C\n Highlights\n Bedrooms: 2\n Bathrooms (full/half): 1/0\n Square footage:\n Rent: $3,700 per month\n Deposit: $5,550\n Parking: 1 Garage Spot Available for $295\n Pets: No Pets.\n Non-smoking property\n Description\n Features Include: - Large eat-in kitchen with ample counter space and cabinet space. Modern cherry cabinetry, granite counters, gas stove, refrigerator, microwave, and dishwasher. - Laundry room attached to kitchen. Full size washer and dryer - 2 storage closets in hallway - Large spacious bathroom with granite from floor to ceiling - 2 Large Bedrooms with great light - Spacious living room with excellent natural light and brick fireplace - Dining room off the living room with french doors. Potential to be a 3rd bedroom. - Hardwood floors - C

In [74]:
sfdf = sfdf[(sfdf['region']=='sfc')]